In [6]:
!pip install -U -q google-genai google-auth

In [98]:
from google import genai
from google.genai import types
import pandas as pd
from google.colab import userdata

In [99]:
from google.colab import userdata
api_key = userdata.get('GEMINI_API_KEY2')
client = genai.Client(api_key=api_key)

In [100]:
# Load Data
holdings_file = client.files.upload(file='/content/holdings.csv')
trades_file = client.files.upload(file='/content/holdings.csv')

In [103]:
def chat_with_data(query):
    system_prompt = (
        "You are a specialized Financial Data Bot. You have access to 'holdings.csv' and 'holdings.csv'. "
        "1. Use the code_execution tool to find answers in these files. "
        "2. When calculating totals for a fund, you must sum ALL relevant rows (both positive and negative) to get the net result. Do not ignore losses. "
        "3. IF THE ANSWER IS NOT FOUND IN THE GIVEN FILES, you MUST respond with: 'Sorry can not find the answer'. "
        "4. Do not use any outside knowledge or the internet to answer. "
        "5. Do not provide explanations if the data is missing; only use the required phrase."
    )

    # We define the tool as a dictionary to avoid the AttributeError
    tools_config = [{'code_execution': {}}]

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        config=types.GenerateContentConfig(
            system_instruction=system_prompt,
            tools=tools_config,
            temperature=0.0
        ),
        contents=[holdings_file, trades_file, query]
    )
    return response.text

In [108]:
def start_chatbot():
    print("--- Financial Data Bot Active ---")
    print("Type your questions below. Type 'exit' or 'quit' to stop.")

    while True:
        user_query = input("\nYou: ")

        # Check if the user wants to quit
        if user_query.lower() in ['exit', 'quit', 'q']:
            print("Chatbot session ended. Goodbye!")
            break

        # Get the response from your existing function
        try:
            answer = chat_with_data(user_query)
            print(f"Bot: {answer}")
        except Exception as e:
            print(f"An error occurred: {e}")

# Call the function to start chatting
start_chatbot()

--- Financial Data Bot Active ---
Type your questions below. Type 'exit' or 'quit' to stop.

You: For the 'Northpoint 401K' portfolio, analyze the asset 'FB':  What is the total quantity traded in the trades file?
Bot: The total quantity traded for the 'FB' asset in the 'Northpoint 401K' portfolio is 35000.0.

You: in which year did india won the world cup?
Bot: Sorry can not find the answer.

You: What is the total Quantity (Qty) for the portfolio 'Garfield' for the security 'EJ0445951'?
Bot: The total Quantity (Qty) for the portfolio 'Garfield' for the security 'EJ0445951' is: 2784588.0.


KeyboardInterrupt: Interrupted by user